In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
import timeit
""


sc= SparkContext()
sqlContext = SQLContext(sc)
data = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('input/AAPL.csv')


data = data.withColumn("Open", data["Open"].cast(DoubleType()))
data = data.withColumn("Date", data["Date"].cast(DoubleType()))
data = data.withColumn("High", data["High"].cast(DoubleType()))
data = data.withColumn("Low", data["Low"].cast(DoubleType()))
data = data.withColumn("Close", data["Close"].cast(DoubleType()))
data = data.withColumn("Adj Close", data["Adj Close"].cast(DoubleType()))
data = data.withColumn("Volume", data["Volume"].cast(DoubleType()))

vectorAssembler = VectorAssembler(inputCols = ['Date','Open','High', 'Low', 'Close', 'Volume'], outputCol = 'features')
vdata=vectorAssembler.setHandleInvalid("skip").transform(data).select(['features','Adj Close'])
splits = vdata.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]


In [2]:
%%timeit

lr = LinearRegression(featuresCol = 'features', labelCol='Adj Close', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
#print("Coefficients: " + str(lr_model.coefficients))
#print("Intercept: " + str(lr_model.intercept))

1.55 s ± 142 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
